# 3장 피마 인디언 당뇨병 예측

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve,roc_curve,f1_score,confusion_matrix
from sklearn.metrics import precision_score, recall_score , roc_auc_score
from sklearn.preprocessing import StandardScaler , Binarizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [2]:
diabetes_data = pd.read_csv('./data/diabetes.csv')
print(diabetes_data['Outcome'].value_counts())
diabetes_data.head(3)

0    500
1    268
Name: Outcome, dtype: int64


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [3]:
x = diabetes_data.loc[:,['Glucose','BloodPressure','BMI', 'Age']]
y = diabetes_data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(x,y,test_size = 0.2 , random_state=156)

# 로지스틱 회귀로 학습, 예측 및 평가 수행
lr = LogisticRegression()
lr.fit(X_train,y_train)

joblib.dump(lr, 'model/diabetes_lr.pkl')

['model/diabetes_lr.pkl']

In [4]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X_train, y_train)
joblib.dump(svc, 'model/diabetes_svm.pkl')

['model/diabetes_svm.pkl']

In [5]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
joblib.dump(dtc, 'model/diabetes_dtc.pkl')

['model/diabetes_dtc.pkl']

In [6]:
model_lr = joblib.load('model/diabetes_lr.pkl')
model_svm = joblib.load('model/diabetes_svm.pkl')
model_dt = joblib.load('model/diabetes_dtc.pkl')

In [7]:
y_pred_lr = model_lr.predict(X_test)
y_pred_svm = model_svm.predict(X_test)
y_pred_dt = model_dt.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score
acc_lr = accuracy_score(y_test, y_pred_lr)
acc_svm = accuracy_score(y_test, y_pred_svm)
acc_dt = accuracy_score(y_test, y_pred_dt)
print(acc_lr, acc_svm, acc_dt)

0.7402597402597403 0.7597402597402597 0.6753246753246753


In [9]:
diabetes_data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [10]:
test_data = np.array([50,72,33.6,50]).reshape(1, 4)

In [11]:
index_lr = model_lr.predict(test_data)[0]
index_svm = model_svm.predict(test_data)[0]
index_dt = model_dt.predict(test_data)[0]

In [12]:
sp_names = ['부정','긍정']
print(sp_names[index_lr], sp_names[index_svm], sp_names[index_dt])

부정 부정 부정


In [13]:
# 딥러닝 모델 설정
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

model=Sequential([
    Dense(100,input_shape=(4,), activation="relu"),
    Dense(80, input_shape=(4,),activation="relu"),
    Dense(50, input_shape=(4,),activation="relu"),
    Dense(20, input_shape=(4,),activation="relu"),
    Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               500       
_________________________________________________________________
dense_2 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_3 (Dense)              (None, 50)                4050      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                1020      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 13,671
Trainable params: 13,671
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


In [14]:
# 모델 컴파일 
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath="model/diabetes_deep.hdf5", 
                               monitor='val_loss', 
                               verbose=1, save_best_only=True)

In [16]:
# 모델 학습
model.fit(X_train, y_train, validation_split=0.2, epochs=500, batch_size=50,
          callbacks=[checkpointer])

Train on 491 samples, validate on 123 samples
Epoch 1/500
491/491 [==============================] - 0s 292us/step - loss: 1.3318 - accuracy: 0.5051 - val_loss: 0.9149 - val_accuracy: 0.6504

Epoch 00001: val_loss improved from inf to 0.91489, saving model to model/diabetes_deep.hdf5
Epoch 2/500
491/491 [==============================] - 0s 54us/step - loss: 0.8693 - accuracy: 0.6130 - val_loss: 0.9395 - val_accuracy: 0.6585

Epoch 00002: val_loss did not improve from 0.91489
Epoch 3/500
491/491 [==============================] - 0s 20us/step - loss: 0.7533 - accuracy: 0.5967 - val_loss: 0.8102 - val_accuracy: 0.6585

Epoch 00003: val_loss improved from 0.91489 to 0.81016, saving model to model/diabetes_deep.hdf5
Epoch 4/500
491/491 [==============================] - 0s 24us/step - loss: 0.6667 - accuracy: 0.6925 - val_loss: 0.5946 - val_accuracy: 0.6423

Epoch 00004: val_loss improved from 0.81016 to 0.59464, saving model to model/diabetes_deep.hdf5
Epoch 5/500
491/491 [==============

491/491 [==============================] - 0s 22us/step - loss: 0.5181 - accuracy: 0.7475 - val_loss: 0.5831 - val_accuracy: 0.7480

Epoch 00041: val_loss did not improve from 0.56251
Epoch 42/500
491/491 [==============================] - 0s 22us/step - loss: 0.5242 - accuracy: 0.7332 - val_loss: 0.5742 - val_accuracy: 0.6829

Epoch 00042: val_loss did not improve from 0.56251
Epoch 43/500
491/491 [==============================] - 0s 22us/step - loss: 0.5235 - accuracy: 0.7475 - val_loss: 0.6189 - val_accuracy: 0.6829

Epoch 00043: val_loss did not improve from 0.56251
Epoch 44/500
491/491 [==============================] - ETA: 0s - loss: 0.4747 - accuracy: 0.76 - 0s 22us/step - loss: 0.5158 - accuracy: 0.7352 - val_loss: 0.6122 - val_accuracy: 0.6748

Epoch 00044: val_loss did not improve from 0.56251
Epoch 45/500
491/491 [==============================] - 0s 20us/step - loss: 0.5268 - accuracy: 0.7312 - val_loss: 0.5824 - val_accuracy: 0.7398

Epoch 00045: val_loss did not improve

491/491 [==============================] - 0s 22us/step - loss: 0.5004 - accuracy: 0.7475 - val_loss: 0.6129 - val_accuracy: 0.6423

Epoch 00082: val_loss did not improve from 0.55520
Epoch 83/500
491/491 [==============================] - 0s 20us/step - loss: 0.5025 - accuracy: 0.7475 - val_loss: 0.6241 - val_accuracy: 0.6585

Epoch 00083: val_loss did not improve from 0.55520
Epoch 84/500
491/491 [==============================] - 0s 21us/step - loss: 0.4704 - accuracy: 0.7719 - val_loss: 0.6283 - val_accuracy: 0.7073

Epoch 00084: val_loss did not improve from 0.55520
Epoch 85/500
491/491 [==============================] - 0s 24us/step - loss: 0.4862 - accuracy: 0.7515 - val_loss: 0.5928 - val_accuracy: 0.7317

Epoch 00085: val_loss did not improve from 0.55520
Epoch 86/500
491/491 [==============================] - 0s 22us/step - loss: 0.4920 - accuracy: 0.7699 - val_loss: 0.5802 - val_accuracy: 0.7236

Epoch 00086: val_loss did not improve from 0.55520
Epoch 87/500
491/491 [======

491/491 [==============================] - 0s 22us/step - loss: 0.4494 - accuracy: 0.7821 - val_loss: 0.6047 - val_accuracy: 0.7317

Epoch 00123: val_loss did not improve from 0.55520
Epoch 124/500
491/491 [==============================] - 0s 22us/step - loss: 0.4661 - accuracy: 0.7617 - val_loss: 0.6328 - val_accuracy: 0.7154

Epoch 00124: val_loss did not improve from 0.55520
Epoch 125/500
491/491 [==============================] - 0s 24us/step - loss: 0.4874 - accuracy: 0.7515 - val_loss: 0.6456 - val_accuracy: 0.7073

Epoch 00125: val_loss did not improve from 0.55520
Epoch 126/500
491/491 [==============================] - 0s 22us/step - loss: 0.4546 - accuracy: 0.7882 - val_loss: 0.6303 - val_accuracy: 0.7398

Epoch 00126: val_loss did not improve from 0.55520
Epoch 127/500
491/491 [==============================] - 0s 20us/step - loss: 0.4583 - accuracy: 0.7800 - val_loss: 0.6617 - val_accuracy: 0.6911

Epoch 00127: val_loss did not improve from 0.55520
Epoch 128/500
491/491 [=

491/491 [==============================] - 0s 22us/step - loss: 0.4585 - accuracy: 0.7841 - val_loss: 0.6373 - val_accuracy: 0.7398

Epoch 00164: val_loss did not improve from 0.55520
Epoch 165/500
491/491 [==============================] - 0s 20us/step - loss: 0.4345 - accuracy: 0.7841 - val_loss: 0.6776 - val_accuracy: 0.6667

Epoch 00165: val_loss did not improve from 0.55520
Epoch 166/500
491/491 [==============================] - 0s 22us/step - loss: 0.4484 - accuracy: 0.7739 - val_loss: 0.6311 - val_accuracy: 0.7236

Epoch 00166: val_loss did not improve from 0.55520
Epoch 167/500
491/491 [==============================] - 0s 20us/step - loss: 0.4637 - accuracy: 0.7780 - val_loss: 0.6409 - val_accuracy: 0.6992

Epoch 00167: val_loss did not improve from 0.55520
Epoch 168/500
491/491 [==============================] - 0s 20us/step - loss: 0.4506 - accuracy: 0.7719 - val_loss: 0.6392 - val_accuracy: 0.7236

Epoch 00168: val_loss did not improve from 0.55520
Epoch 169/500
491/491 [=

...

491/491 [==============================] - 0s 20us/step - loss: 0.3801 - accuracy: 0.8289 - val_loss: 0.6356 - val_accuracy: 0.7805

Epoch 00328: val_loss did not improve from 0.55520
Epoch 329/500
491/491 [==============================] - ETA: 0s - loss: 0.4080 - accuracy: 0.82 - 0s 20us/step - loss: 0.3508 - accuracy: 0.8452 - val_loss: 0.7020 - val_accuracy: 0.6992

Epoch 00329: val_loss did not improve from 0.55520
Epoch 330/500
491/491 [==============================] - 0s 22us/step - loss: 0.4029 - accuracy: 0.8004 - val_loss: 0.7399 - val_accuracy: 0.6992

Epoch 00330: val_loss did not improve from 0.55520
Epoch 331/500
491/491 [==============================] - 0s 20us/step - loss: 0.3968 - accuracy: 0.8187 - val_loss: 0.7419 - val_accuracy: 0.6992

Epoch 00331: val_loss did not improve from 0.55520
Epoch 332/500
491/491 [==============================] - 0s 21us/step - loss: 0.3831 - accuracy: 0.8228 - val_loss: 0.7133 - val_accuracy: 0.7154

Epoch 00332: val_loss did not imp

491/491 [==============================] - 0s 22us/step - loss: 0.3537 - accuracy: 0.8248 - val_loss: 0.7230 - val_accuracy: 0.7236

Epoch 00369: val_loss did not improve from 0.55520
Epoch 370/500
491/491 [==============================] - 0s 20us/step - loss: 0.3529 - accuracy: 0.8350 - val_loss: 0.7080 - val_accuracy: 0.7642

Epoch 00370: val_loss did not improve from 0.55520
Epoch 371/500
491/491 [==============================] - 0s 22us/step - loss: 0.3477 - accuracy: 0.8411 - val_loss: 0.7185 - val_accuracy: 0.7317

Epoch 00371: val_loss did not improve from 0.55520
Epoch 372/500
491/491 [==============================] - 0s 22us/step - loss: 0.3703 - accuracy: 0.8330 - val_loss: 0.7548 - val_accuracy: 0.7236

Epoch 00372: val_loss did not improve from 0.55520
Epoch 373/500
491/491 [==============================] - 0s 20us/step - loss: 0.3745 - accuracy: 0.8350 - val_loss: 0.7427 - val_accuracy: 0.7073

Epoch 00373: val_loss did not improve from 0.55520
Epoch 374/500
491/491 [=

491/491 [==============================] - 0s 20us/step - loss: 0.3643 - accuracy: 0.8330 - val_loss: 0.7384 - val_accuracy: 0.7073

Epoch 00410: val_loss did not improve from 0.55520
Epoch 411/500
491/491 [==============================] - 0s 22us/step - loss: 0.3499 - accuracy: 0.8310 - val_loss: 0.7238 - val_accuracy: 0.7317

Epoch 00411: val_loss did not improve from 0.55520
Epoch 412/500
491/491 [==============================] - 0s 22us/step - loss: 0.3460 - accuracy: 0.8473 - val_loss: 0.6756 - val_accuracy: 0.7317

Epoch 00412: val_loss did not improve from 0.55520
Epoch 413/500
491/491 [==============================] - 0s 20us/step - loss: 0.3459 - accuracy: 0.8493 - val_loss: 0.7165 - val_accuracy: 0.7073

Epoch 00413: val_loss did not improve from 0.55520
Epoch 414/500
491/491 [==============================] - 0s 22us/step - loss: 0.3482 - accuracy: 0.8371 - val_loss: 0.7584 - val_accuracy: 0.7480

Epoch 00414: val_loss did not improve from 0.55520
Epoch 415/500
491/491 [=

491/491 [==============================] - 0s 22us/step - loss: 0.3768 - accuracy: 0.8248 - val_loss: 0.8014 - val_accuracy: 0.6829

Epoch 00451: val_loss did not improve from 0.55520
Epoch 452/500
491/491 [==============================] - 0s 22us/step - loss: 0.3890 - accuracy: 0.8187 - val_loss: 0.7203 - val_accuracy: 0.6829

Epoch 00452: val_loss did not improve from 0.55520
Epoch 453/500
491/491 [==============================] - 0s 22us/step - loss: 0.3834 - accuracy: 0.8147 - val_loss: 0.6816 - val_accuracy: 0.6992

Epoch 00453: val_loss did not improve from 0.55520
Epoch 454/500
491/491 [==============================] - 0s 22us/step - loss: 0.3807 - accuracy: 0.8208 - val_loss: 1.1749 - val_accuracy: 0.6585

Epoch 00454: val_loss did not improve from 0.55520
Epoch 455/500
491/491 [==============================] - 0s 22us/step - loss: 0.4414 - accuracy: 0.8187 - val_loss: 0.6600 - val_accuracy: 0.6829

Epoch 00455: val_loss did not improve from 0.55520
Epoch 456/500
491/491 [=

491/491 [==============================] - 0s 22us/step - loss: 0.3472 - accuracy: 0.8432 - val_loss: 0.7295 - val_accuracy: 0.7073

Epoch 00492: val_loss did not improve from 0.55520
Epoch 493/500
491/491 [==============================] - 0s 22us/step - loss: 0.3423 - accuracy: 0.8289 - val_loss: 0.6858 - val_accuracy: 0.7317

Epoch 00493: val_loss did not improve from 0.55520
Epoch 494/500
491/491 [==============================] - 0s 20us/step - loss: 0.3235 - accuracy: 0.8574 - val_loss: 0.7576 - val_accuracy: 0.6992

Epoch 00494: val_loss did not improve from 0.55520
Epoch 495/500
491/491 [==============================] - 0s 22us/step - loss: 0.3035 - accuracy: 0.8656 - val_loss: 0.7184 - val_accuracy: 0.7317

Epoch 00495: val_loss did not improve from 0.55520
Epoch 496/500
491/491 [==============================] - 0s 22us/step - loss: 0.2960 - accuracy: 0.8656 - val_loss: 0.7422 - val_accuracy: 0.7154

Epoch 00496: val_loss did not improve from 0.55520
Epoch 497/500
491/491 [=

In [17]:
from keras.models import load_model
model_deep = load_model("model/diabetes_deep.hdf5")

In [18]:
print("\n Accuracy : %.4f"%(model.evaluate(X_test,y_test,verbose=2))[1])


 Accuracy : 0.6818


In [44]:
test_data = np.array([148.0,72,33.6,50]).reshape(1, 4)

In [47]:
test_data = np.array([85.0,66.0,26.6,31]).reshape(1, 4)
test_data

array([[85. , 66. , 26.6, 31. ]])

In [21]:
x.head()

,Glucose,BloodPressure,BMI,Age
0,148,72,33.6,50
1,85,66,26.6,31
2,183,64,23.3,32
3,89,66,28.1,21
4,137,40,43.1,33


In [48]:
model_deep.predict(test_data)

array([[0.377059]], dtype=float32)

In [49]:
print(sp_names[model_deep.predict_classes(test_data)[0][0]])

부정
